## Evaluation Process 

To evaluate a model using raw data, one needs to convert the data to the supported `hdf5` file format. Although the [Data Generation](1.%20Data%20Generation.ipynb) process requires labels, pseudo-labels can be inserted (e.g. use all zeros). The evaluation process will output 
* the scores for each label (you can ignore if you used pseudo-labels), and 
* the predictions for each file, which are also saved in `.json` format.

Example follows

In [ ]:
import numpy as np

from pathlib import Path
from end2you.evaluation_process import EvaluationProcess
from end2you.utils import Params

We use the class `Params` to elegantly define the parameters required for the evaluation process.
The parameters for the evaluation process are the following:
```
- log_file: Path to save log file
- metric      : Metric to use for evaluation. One of [`ccc`, `uar`, `mse`].
- dataset_path: Path to evaluation .hdf5 files.
- cuda        : Load data in cuda (should be same as in train).
- num_workers : Number of workers to use to load data.
- modality    : Modality to be used for. Values `audio`, `visual`, `audiovisual`.
- root_dir    : Path to save the output files of end2you.
- num_gpus    : Number of GPUs to use. Need to have more than one defined with `CUDA_VISIBLE_DEVICES`.
- seq_length  : Number of consecutive frames to load per batch.
- ckpt_path   : Path to checkpoint. Needs to be defined.
- model       : The parameters of the model.
- num_outs    : The number of outputs of the model.
```

Example:

In [ ]:
params = Params(dict_params={
    'metric': 'ccc',
    'modality': 'audio',
    'dataset_path':'/path',
    'model_path':'/path/to/saved/model/last.pth.tar',
    'cuda':False,
    'num_gpus':1,
    'root_dir':'.',
    'seq_length': 150,
    'num_workers':2,
    'log_file': 'evaluation.log',
    'model':Params(dict_params={'model_name':'emo18', 'num_outs':3})
})

Defining the parameters we can now invoke the EvaluationProcess class and start the evaluation.
For example

In [ ]:
eval_process = EvaluationProcess(params)
scores, predictions = eval_process.start()

`eval_process` saves the predictions of each file in the `root_dir/predictions.json` file and returns the following two dictionaries : 
    - `scores`     : contains as keys the labels and the values are the correspoding score
                     (if pseudo-labels are used ignore this).
    - `predictions`: contains as keys the names of the files and the values are the predictions of the model
                     on each file. In addition, it contains a key named 'label_names' with the label names
                     that are predicted.
       E.g. {
          'file1': np.array([[0.2, 0.1], [-0.1, 0.5], [0.3, 0.4]]),
          'file2': np.array([[0.4, 0.3], [-0.5, 0.2], [0.1, -0.2]]),
          ...
          'label_names: array(['arousal', 'valence'])
       }